In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm

In [2]:
from lifelines import CoxPHFitter

In [3]:
record_path = "./records/test_record_1/"

In [4]:
people_df = pd.read_csv(record_path + "/people.csv")

In [5]:
deaths_df = pd.read_csv(record_path + "/deaths.csv")

In [6]:
deaths_df = deaths_df.rename(columns={"Unnamed: 0": "id"})

In [7]:
study_duration = (datetime(2020, 5, 16, 0, 0) - datetime(2020, 2, 1, 0, 0)).days

In [8]:
people_df_died = list(np.zeros(len(people_df)))
people_df_days_dead = list(np.ones(len(people_df)) * study_duration)
for idx, row in tqdm(deaths_df.iterrows(), total = len(deaths_df)):
    people_df_died[idx] = 1
    days_dead = (datetime.strptime(row["timestamp"], "%Y-%m-%d") - datetime(2020, 2, 1, 0, 0)).days
    if days_dead < study_duration:
        people_df_days_dead[idx] = days_dead

100%|██████████| 56587/56587 [00:04<00:00, 11837.77it/s]


In [9]:
people_df["died"] = list(np.array(people_df_died).astype(int))
people_df["days"] = list(np.array(people_df_days_dead).astype(int))

In [10]:
people_df["sex"][people_df["sex"] == "f"] = 0
people_df["sex"][people_df["sex"] == "m"] = 1

/Users/josephbullock/Documents/covidmodelling/JUNE-private/covidmodelling-private/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/josephbullock/Documents/covidmodelling/JUNE-private/covidmodelling-private/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
people_df["ethnicity"][people_df["ethnicity"] == "A"] = 0
people_df["ethnicity"][people_df["ethnicity"] == "B"] = 1
people_df["ethnicity"][people_df["ethnicity"] == "C"] = 2
people_df["ethnicity"][people_df["ethnicity"] == "D"] = 3
people_df["ethnicity"][people_df["ethnicity"] == "E"] = 4

/Users/josephbullock/Documents/covidmodelling/JUNE-private/covidmodelling-private/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/josephbullock/Documents/covidmodelling/JUNE-private/covidmodelling-private/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/josephbullock/Documents/covidmodelling/JUNE-private/covidmodelling-private/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [12]:
people_df.head()

,id,age,socioeconomic_index,primary_activity_id,residence_id,area_id,sex,ethnicity,primary_activity_type,residence_type,died,days
0,12762019,0,3,0,5225984,40080,0,0,None,household,1,37
1,12762020,1,3,0,5225993,40080,1,0,None,household,1,39
2,12762021,1,3,0,5225986,40080,1,0,None,household,1,44
3,12762022,1,3,0,5225984,40080,1,0,None,household,1,50
4,12762023,2,3,4260,5226005,40080,0,0,school,household,1,51


In [13]:
people_cox_df = people_df[["age", "sex", "ethnicity", "died", "days"]]

In [45]:
people_cox_b = people_cox_df[people_cox_df["ethnicity"].isin([0,1])]
people_cox_c = people_cox_df[people_cox_df["ethnicity"].isin([0,2])]
people_cox_d = people_cox_df[people_cox_df["ethnicity"].isin([0,3])]
people_cox_e = people_cox_df[people_cox_df["ethnicity"].isin([0,4])]

In [46]:
people_cox_c["ethnicity"][people_cox_c["ethnicity"] == 2] = 1
people_cox_d["ethnicity"][people_cox_d["ethnicity"] == 3] = 1
people_cox_e["ethnicity"][people_cox_e["ethnicity"] == 4] = 1

/Users/josephbullock/Documents/covidmodelling/JUNE-private/covidmodelling-private/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/josephbullock/Documents/covidmodelling/JUNE-private/covidmodelling-private/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/josephbullock/Documents/covidmodelling/JUNE-private/covidmodelling-private/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [47]:
cph = CoxPHFitter()

In [ ]:
cph.fit(people_cox_b, duration_col="days", event_col="died", show_progress=True)

In [ ]:
cph.print_summary()

In [ ]:
cph = CoxPHFitter()

In [ ]:
cph.fit(people_cox_c, duration_col="days", event_col="died", show_progress=True)

In [ ]:
cph.print_summary()

In [ ]:
cph = CoxPHFitter()

In [ ]:
cph.fit(people_cox_d, duration_col="days", event_col="died", show_progress=True)

In [ ]:
cph.print_summary()

In [ ]:
cph = CoxPHFitter()

In [ ]:
cph.fit(people_cox_e, duration_col="days", event_col="died", show_progress=True)

In [ ]:
cph.print_summary()

In [ ]:
ethnicity_opensafely = {
    "White": [1.,1.,1.]
    "Mixed": [1.11,1.43,1.84]
    "South_Asian": [1.32,1.45,1.58] # note the definition is slightly different
    "Black": [1.29,1.48,1.69]
    "Other": [1.1,1.33,1.61]
}
'''
A=white
B=mixed
C=asian
D=Black
E=other
'''